# Acrobot-v1 DQN

The acrobot environment comes as part of  [OpenAI Gym](https://gym.openai.com/).
The acrobot system includes two joints and two links, where the joint between the two links is actuated. Initially, the links are hanging downwards, and the goal is to swing the end of the lower link up to a given height. This environment contains a continuous state space but a discrete actions space.
The state space of this environment is multidimensional. Most of the dimensions range from -1 to 1 (these dimensions represent the position of the two joints) and some other dimensions (the last 2) with a wider range.

This will be my own try to solve the Acrobot problem using Deep Reinforcement Learning or DQNs.

## Let's write a bit about Q-Learning 

Remember that Q-Learning algorithm is derived from the TD algorithm SARSA.

In the _SARSA_ algorithm, we first initialize the environment (reset it). After initializing the environment we will get a first state S - the initial state. The agent chooses an action A from the actions space of the current state - this choice is done following the policy $\pi$ which in many cases is randomly initialized. Then the agent takes the choosen action A and it observes th returned reward *R* and the next state S'. Once the agen has observed this new state and reward it uses again the policy $\pi$ to choose another action A'.
_SARSA_ is an on-policy algorithm, which means that the policy being followed, is also the policy being learned (modified). In other words, we are updating the same policy that we are following to carry out actions. Sarsa normally works very well and converges fast because we are using the most updated policy to take each decision. But it has a drawback: the policy learned and the policy followed are intimately tied to each other. 
We would like to be able to follow a policy for exploratory purposes, while learning a more optimal (different) policy, so we need an off-policy algorithm.

A good candidate for an off-policy algorithm is Q-Learning.
It is an off policy variant of TD learning and we want to adapt it to work with function approximation (function approximation is not covered here).
How is this adaptation done?

1. Initialize the parameters W arbitrarily. Also define a policy $\pi$.

2. When resetin the environment, the initial state S will be returned to the agent.

3. The agent will do the followin over multiple episodes (an episode ends when the environment returns to the agent a terminal state S): The agent will use the $\epsilon$-greedy policy to take an action A and it will observe (once the action has been taken) the reward R and the next state S' (so far we have taken the steps _SARS_ - the last action is still missing at this point ).
4. The agent will update the W parameters with the following update rule:

$$(1)   {\triangle}W = \alpha(R + \gamma{max _{a}\hat{q}(S', a, w)} - \hat{q}(S, A, w))\nabla_{w}\hat{q}(S, A, w)$$


This equation will be our update rule. As it can be seen, there is a difference between the update rule of _SARSA_ and this update rule. Instead of picking the next action from the same $\epsilon$-greedy policy, the agen will choose an action greedely, meaning that it will choose the action that results in the maximum possible inmediate reward (of course, this action belongs to the action space available for state S'). This will maximize the expected value going forward. It is important to note, that we are *not* taking this action, we are just picking it and using it to update our Wparameters. This is the reason why *Q-Learning* is an off-policy algorithm. This algorithm uses the $\epsilon$-greedy policy to take actions, while it updates the W parameters using a greedy policy.

Thes characteristic of *Q-Learning* gives us the possibility of building on top of it. For example, we could build variation of this algorithm, say to use a more exploratory policy while acting and still learn the optimal value function. This algorithm performs badly when applied or run online, but at any given point we can stop the algorithm (stop exploring) and follow the optimal policy that the agent learnt.
It could also be possible for such algorithm, to learn by "observing" the actions taken by a human. In such approach, the human would be performing the $\epsilon$-greedy choosing of actions, while the algorithm would be learning an optimal policy.

This algorithm could also learn offline or in batches. That is because it is not necessary that the policy update is performed at every step. This is critical for reliably training nerual networks for RL.
Cons: Both *SARSA* and *Q-Learning* do not converge on the global optimum when using a non-linear function approximation. 

## We come now to Deep Q Networks

Deep Q Networs are really an agent at whose heart lies a deep neural network that acts as a function approximator - remember that this neural networks optimizes some 'W' parameters using gradient descents.
Is not the purpose of Function Approximation to optimize some 'W' parameters that represent our feature weights? Remember that we use this parameter vector W to shape an underlying function that help us capture or approximate to the true state value or action value function. And if you remember, what we initially were looking for with the function approximation approach was to change such 'W' parameters - the feature weights -, on small steps away from an error in specific (correct) direction, and we used gradient descent for this purpose.

So, we can then use deep neural networks to do this job for us.
So we can design our deep neural network with different layers, different loss measurements and optimizations, but training such a network, as already known, requires a lot of data and it is not guaranteed that it wil lever converge on the optimal value function. In fact, there are situations where the network weights can oscillate or diverge, due to the high correlation between actions and states, resulting in an unstable - not to mention ineffective - policy. To overcome these problems, researches cam up with several techniques that slightly modifies the base Q-Learning algorithm. Two of these techniques are:

    *Experience Replay
    * Fixed Q Targets

### About Experience Replay

While our agent progresses in its task, it learns from the obtained rewards. But once the agent has learned from it it discards the "experienced" set of state-action-reward-nextState.
The agen could possibly learn more from these experienced set (or tuple) if it could store such experiences to be replayed again at a later point. Especially when we consider that there could be some states that are pretty rare to come by, or some actions that can be pretty costly - we want to learn from these the most. To achieve this, we can create a replay buffer, in which the agent will store each experienced tuple. At a later point in time the agent could sample a small batch of tuples from the replay buffer and learn from it again. So the agent will be able to learn from these experienced tuples multiple times, recall rar occurrences and in general make better use of fire experience.
Let's also think about the relation between the states and the agent's previous action. Every action A$_t$ affects the next state S$_t$ in some way. This means that a sequence of experienced tuples can be highlycorrelated. A naive *Q-Learning* approach that learns from each of these experiences in wequential order rusn the risk of getting swayed by the effercts of this correlation.
Our replay buffer can help us to break such correlation when we sample from this buffer randomly - the sampled batches does not necessarily need to contain sequences of tuples as they were experienced the first time. This prevents action valus from oscillate or diverge - catastrophe.
Note that this approach help us to reduce the reinforcement learning problem -or at least the value learning portion of it- to a **supervised learning** scenario. We could then apply our supervised learning techniques through reinforcement learning.

### About Fixed Q Targets

*Q-Learning* (as we have seen it so far) presents another correlation problem. Remember that *Q-Learning* is a form of Temporal Difference learning. In the equation number one, we have that our **TD target** is:
$$(2)   R + \gamma{max _{a}\hat{q}(S', a, w)}$$

And the goal is to reduce the difference between this TD target and the currently predicted Q-value:
$$(3)   \hat{q}(S, A, w)$$

This difference - the difference between (2) and (3) is called the **TD Error**.

The problem is that our *TD target* depends on the W parameters.
Normally we do not have a problem with this, because every update results in a small change to the parameters and becuase in general we are "moving" in the right direction. If we would set $\alpha$ to one and leap towards the target, then we would overshoot and land in the wrong place. This could affect learning significantly when we use function approximation because all the Q-values are intrinsically tied together through the functino parameters W.
So, the existing correlation in this case, is between the target and the parameters we are changing. Basically we are trying to change the parameteres by making small updates to them, and at the same time these parameter's values are changing! (we are chasing a moving target). We need to decouple our target from the actions our agent takes. This would result in a more stable learning environment. To do that, we need to fix the the function parameters W that we use to generate our target. These fixed parameters will be indicated by $w^-$. These fixed parameters will be used to generate the TD target while the agent will be changing the $w$ parameters for a certain number of learning steps. So, basically, $w^-$ is a copy of $w$. We do not update this copy during the learning steps. Then, at some point we update $w^-$, with the latest $w$ and again, learn from a number of steps and so and so on. This will clearly separate our  TD target from the parameters, making the learning process more stable and less likely to diverge or fall into ascillations.
Our new update rool looks like this:
$$(4)   {\triangle}W = \alpha(R + \gamma{max _{a}\hat{q}(S', a, w^-)} - \hat{q}(S, A, w))\nabla_{w}\hat{q}(S, A, w)$$

## ... and now Deep Q-Learning

The Deep Q.Learning algorithm have two main process. One of them samples the environment by performing actions and it sotres all the observed experienced tuples in a replay memory. The other process selects a batch of the stored tuples and allow the agent to learn from that batch using the gradient descent update step we reviewed lines above. These two process are not depending on each other. This means that we could do multiple sampling steps, then we could do one learning step, or we could do several learning steps with different random batches. The rest of the algorithm supports these steps. 
What the algorithm should do at first is initialize an empty memory replay (normally a circular buffer memory where all the most recent experienced tuples are saved). Then, also the parameter weights of the neural network must be initialized. There are certain best practices to do this. One of this practices consist in sample the weights randomly from a normal distribution with variance equal to $2 * number of inputs to each neuron$.
Now, if we also want to use *Fixed Q-Targets* technique, we sould need a second set of parameters (i.e. $w^-$) which we can initialize to $w$.

## Now, the implementation

In this environment, the reward goes hight when both the links are up - so the agent must learn how much to rotate the joints and in which directions. This means the agent needs to develop some sort of intuition. In order to logically represent this intuition and train it, we need to express this as a formula that we can optimize on. The loss is just a value that indicates how far our prediction is from the actual target.
For example, the prediction of the model could indicate that it sees more value in turning the joints in a direction that points the links downwards when in fact it can gain more reward by turning the joints in a different direction to make the links point upwards. We want to decrease this gap between the prediction and the target (loss). We will define our loss function as follows:
$$loss = (R + \gamma{max_{a}\hat{q}(S', a, w^-)} - \hat{q}(S, A, w))^2$$

We first carry out an action A, and observe the reward R and resulting new state S'. Based on the result, we calculate the maximum target Q and then discount it so that the future reward is worth less than immediate reward (It is a same concept as interest rate for money. Immediate payment always worth more for same amount of money). Lastly, we add the current reward to the discounted future reward to get the target value. Subtracting our current prediction from the target gives the loss. Squaring this value allows us to punish the large loss value more and treat the negative values same as the positive values.

If we use keras (we will), then we only need to define the TD target - keras will take care of the most difficult part, which is substracting the target from the Neural Network output and calculates the square of it. The learning rate (i.e. $\alpha$ - we define it when creating the Neural Network model) will also be applied by keras.

In [ ]:
# libraries used
import random
import gym
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque

import matplotlib.pyplot as plt

# Set plotting options
%matplotlib inline
plt.style.use('ggplot')
np.set_printoptions(precision=3, linewidth=120)

In [ ]:
# let's create the environment

acrobot_env = gym.make('Acrobot-v1')
acrobot_env.seed(1981)

# explorint the state (observation) space
print("State space: ", acrobot_env.observation_space)
print(" - low:   ", acrobot_env.observation_space.low)
print(" - high: ", acrobot_env.observation_space.high)

# explorint the action space
print("Action space: ", acrobot_env.action_space)

In [ ]:
# Setting hyper-parameters and general variables
state_size = acrobot_env.observation_space.shape[0]
action_size = acrobot_env.action_space.n
memory = deque(maxlen=5000)
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
batch_size = 64
number_episodes=5000
done = False

In [ ]:
# we need to define a method that helps us storing the experienced tuples in our memory
def remember_experience(state, action, reward, nextState, done):
    memory.append((state, action, reward, nextState, done))

In [ ]:
def act(state):
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    
    action_picked = model.predict(state)
    # return the action that produces the most reward
    return np.argmax(action_picked[0])

In [ ]:
def replay(batch_size, epsilon, epsilon_min, epsilon_decay):
    batch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in batch:
        target = reward
        if not done:
            target = reward + (gamma * np.amax(model.predict(next_state)[0]))
            target_aux = model.predict(state)
            target_aux[0][action] = target
            model.fit(state, target_aux, epochs=1, verbose=0)
            
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

In [ ]:
def save(name):
    model.save_weights(name)
    
def load(name):
    model.load_weights(name)

In [ ]:
# Creating our Neural Net for Deep-Q learning model
model = Sequential()
model.add(Dense(24, input_dim=state_size, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(action_size, activation='linear'))
model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
load("./cartpole-dqn.h5")

In [ ]:
for  eps in range(number_episodes):
    state = acrobot_env.reset()
    state = np.reshape(state, [1, state_size])
    time_step = 0
    done = False
    while not done:
        # acrobot_env.render()
        action = act(state)
        next_state, reward, done, _ = acrobot_env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        remember_experience(state, action, reward, next_state, done)
        state = next_state
        time_step += 1
        
        if done:
            print("episodes: {}/{}, socre: {}, e: {:.2}"
                 .format(eps, number_episodes, time_step, epsilon))
            acrobot_env.close()
            #break
            
    # do not learn unless there is enough amounts of experience stored
    # in memory to learn from
        if len(memory) > batch_size:
            replay(batch_size, epsilon, epsilon_min, epsilon_decay)
    if eps % 10 == 0:
        save("./cartpole-dqn.h5")